Importation du fichier de donnée retraité durant l'étape de datavisualisation

In [25]:
import pandas as pd
import numpy as np

df = pd.read_csv("Data Satisfaction - Dan processed.csv")

df.head()

,Unnamed: 0,Commentaire,star,date,client,reponse,source,company,ville,date_commande,ecart,Language,Commentaires_reduit,com_nb_mots,Commentaires_reduit_racine,Commentaires_reduit_4mots
0,0,"Bonjour , Ca doit faire 5 ans environ que je s...",1,2021-06-20 00:00:00+00:00,AUDREY Du 62,NaN,TrustPilot,ShowRoom,NaN,NaN,NaN,fr,"bonjour , ca doit 5 ans environ membre showroo...",246,"bonjour , ca doit 5 an environ membr showroopr...",bonjour doit environ membre showrooprive jamai...
1,1,Vente lacoste article manquant photo prise sur...,1,2021-06-20 00:00:00+00:00,Nanasky De Verteuil,NaN,TrustPilot,ShowRoom,NaN,NaN,NaN,fr,vente lacoste manquant photo prise 6 articles ...,20,vent lacost manqu photo pris 6 articl moiti li...,vente lacoste manquant photo prise articles mo...
2,2,"Vente Lacoste Honteuse , article erroné , arti...",1,2021-06-19 00:00:00+00:00,Vanessa L,NaN,TrustPilot,ShowRoom,NaN,NaN,NaN,fr,"vente lacoste honteuse , erroné , manquant , b...",219,"vent lacost honteux , erron , manqu , bon livr...",vente lacoste honteuse erroné manquant livrais...
3,3,J'ai commandé des mules de la marque Moosefiel...,2,2021-06-19 00:00:00+00:00,Valery PERRAULT,"Bonjour , Je suis sincèrement navré d'apprendr...",TrustPilot,ShowRoom,NaN,NaN,NaN,fr,"commandé mules marque moosefield , très déçue ...",29,"command mul marqu moosefield , tres déçu produ...",commandé mules marque moosefield produit étiqu...
4,4,Commande téléphone etat A+ . Livraison d un vi...,1,2021-06-19 00:00:00+00:00,JULIE DRINGENBERG,"Bonjour Julie , Je suis sincèrement désolé de ...",TrustPilot,ShowRoom,NaN,NaN,NaN,fr,téléphone etat a+ . livraison vieux téléphone ...,52,téléphon etat a+ . livraison vieux téléphon po...,téléphone etat livraison vieux téléphone pourr...


Retraitement du fichier:

- Attribution valeur 1 ou 0 suivant la présence de données ou de NAN pour les colonnes client, ville et réponse
- suppression de la colonne langue, ansin que de la colonne Unnamed:0

In [2]:
df = df.drop(["Unnamed: 0","Language"],axis =1)

In [3]:
df["client"] = df["client"].isna().replace(to_replace =[False,True], value=[1,0])
df["ville"] = df["ville"].isna().replace(to_replace =[False,True], value=[1,0])
df["reponse"] = df["reponse"].isna().replace(to_replace =[False,True], value=[1,0])

- Attribution de valeur numérique pour les colonnes source et company

In [4]:
df["source"] = df["source"].isna().replace(to_replace =["TrustPilot","TrustedShop"], value=[1,0])
df["company"] = df["company"].isna().replace(to_replace =["ShowRoom","VeePee"], value=[1,0])

## Modelisation 1

Dans une premiere tentative de modélisation, nous allons appliquer un algorithme de gradient boosting sur la colonnes: "commentaires_reduit_4mots".
Pour une optimisation des temps de calculs, nous allons vectoriser la colonne commentaire réduit 4 mots avec les mots revenant au moins 100 fois dans les commentaires
#### La notions de date ne sera pas pris en compte pour des raisons de facilités.

- Suppression des colonnes autres que "commentaires_réduit_4mots" et "star"

In [5]:
df1 = df[["star", "Commentaires_reduit_4mots"]]

Verification et suppression des valeurs NaN

In [6]:
df1.isna().sum()

star                          0
Commentaires_reduit_4mots    35
dtype: int64

In [7]:
df1 = df1.dropna(axis = 0, how = "all", subset =["Commentaires_reduit_4mots"])

Determination de l'ensemble des mots apparaissant plus d'une centaine de fois dans les commentaires

In [8]:
from collections import Counter
chaine = ' '.join(str(i).lower() for i in df1.Commentaires_reduit_4mots)
dico = Counter(chaine.split())
freqmots = pd.DataFrame.from_dict(dico, orient="index", columns = ["iteration"])
freqmots = freqmots.sort_values("iteration", ascending = False)
freq_sup_100 = freqmots[freqmots["iteration"] >= 100].reset_index()


In [9]:
freq_sup_100.head()

,index,iteration
0,livraison,4945
1,plus,4814
2,colis,3400
3,site,3173
4,service,3161


On vectorise à present l"ensemble des commentaires par cette ensemble

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit_transform(freq_sup_100["index"])
tokenized = vectorizer.vocabulary_
X = vectorizer.transform(df1["Commentaires_reduit_4mots"]).todense()

On applique à present l'algorithme de GradientBoosting:

In [11]:
y = df1["star"]

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100)

In [13]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

y_pred = clf.predict(X_test)

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [14]:
pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"])

valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,772,34,32,50,74
2,101,19,25,47,52
3,64,14,35,127,107
4,33,9,29,214,338
5,28,5,9,96,1007


In [15]:
clf.score(X_test,y_test)

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.616380608250527

on obtient un score R² de 0.62, et on observe que ce sont surtout les valeurs extremes (1 & 5) qui sont en majeur partie prédit correctement

### Optimisation : variations des parametres

Pour optimiser les performances du model, une premiere piste serait de faire varier : 
 - Les parametres du models
 - le nombre de mots utilisé dans la vectorisation des commentaires

Une estimation exhaustive de la performance en fonction des parametres ainsi que du nombre de mots à travers le code ci dessous n'as pas été possible en raison de temps de calcul extremement long (supérieur à 12h)

In [ ]:
#from sklearn.model_selection import GridSearchCV
#mots = [25,50,75,100]
#parametres = {"n_estimators":[50,75,100,125,150],"learning_rate": [0.1,0.5,1,1.5,2],"max_depth":[1, 2, 3,4,5]}
#gridcv = {}
#for i in mots: 
#    freq = freqmots[freqmots["iteration"] >= i].reset_index()
#    vectorizer.fit_transform(freq["index"])
#    tokenized = vectorizer.vocabulary_
#    X = vectorizer.transform(df1["Commentaires_reduit_4mots"]).todense()
#    X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100) 
#    gcv=GridSearchCV(estimator = clf, param_grid = parametres,cv=3)
#    gcv.fit(X_train,y_train)
#    gridcv[i]= gcv

Une approche itérative a été préféré pour voir si la variation d'un des parametres améliore les performances du model

In [46]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=2, random_state=0).fit(X_train, y_train)

y_pred = clf.predict(X_test)
display(pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"]))
print(clf.score(X_test,y_test))


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,756,56,37,60,53
2,99,23,22,64,36
3,64,25,41,135,82
4,46,10,29,231,307
5,26,8,11,105,995


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6160794941282746


In [47]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0).fit(X_train, y_train)

y_pred = clf.predict(X_test)
display(pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"]))
print(clf.score(X_test,y_test))

C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,757,63,44,48,50
2,97,32,35,49,31
3,66,30,53,125,73
4,45,12,48,221,297
5,20,10,19,131,965


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6106594399277326


La variation de la profondeur ne semble pas améliorer significativement la performance du model. Voyons ce qu'il en est pour le nombre d'estimateurs:

In [48]:
clf = GradientBoostingClassifier(n_estimators=125, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

y_pred = clf.predict(X_test)
display(pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"]))
print(clf.score(X_test,y_test))

C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,772,34,31,52,73
2,102,19,26,44,53
3,64,14,35,129,105
4,35,9,32,210,337
5,25,5,8,102,1005


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6145739235170129


In [49]:
clf = GradientBoostingClassifier(n_estimators=150, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

y_pred = clf.predict(X_test)
display(pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"]))
print(clf.score(X_test,y_test))

C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,772,34,31,52,73
2,102,19,26,44,53
3,64,14,35,129,105
4,35,9,32,210,337
5,25,5,8,102,1005


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6145739235170129


La variation des estimateurs ne semble pas non plus avoir un grand impact sur la performance du model.

Faisons à présent varier le nombres de mot utilisé dans la vectorisation des commentaires:

In [51]:
freq = freqmots[freqmots["iteration"] >= 75].reset_index()

vectorizer.fit_transform(freq["index"])
tokenized = vectorizer.vocabulary_
X = vectorizer.transform(df1["Commentaires_reduit_4mots"]).todense()

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100)

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

y_pred = clf.predict(X_test)
display(pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"]))
print(clf.score(X_test,y_test))


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,770,33,25,64,70
2,100,18,25,56,45
3,64,11,45,125,102
4,37,5,27,211,343
5,29,4,10,92,1010


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6184884071062933


In [52]:
freq = freqmots[freqmots["iteration"] >= 50].reset_index()

vectorizer.fit_transform(freq["index"])
tokenized = vectorizer.vocabulary_
X = vectorizer.transform(df1["Commentaires_reduit_4mots"]).todense()

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100)

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

y_pred = clf.predict(X_test)
display(pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"]))
print(clf.score(X_test,y_test))

C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,781,32,27,48,74
2,95,23,23,53,50
3,61,20,44,119,103
4,36,7,31,213,336
5,26,5,9,83,1022


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.6272207166516109


La diminution du seuil de fréquence pour la prise en compte d'un mot dans la vectorisation semble avoir un plus grand impact que les variations des parametres du model. L'impact sur le R² reste cependant faible.

#### Observations:
Notre premiere piste consistant à faire varier Les parametres du models ainsi que le nombre de mots utilisé dans la vectorisation des commentaires, n'a pas permis d'améliorer de maniere significative la modelisation.

### Optimisation : rééchantillonage

En observant les matrices de confusions de nos précédentes itérations, on note que ce sont principalement les valeurs extremes (1 - 5) qui sont le mieux attribués. Si l'on se réfere à l'analyse exploratoire des données que nous avons effectué précédemment, ces notes extremes sont sur-representées par rapport aux notes intermédiaires.

Essayons de procéder à un rééchantillonnage des données avant d'appliquer notre model.


In [27]:
from imblearn.metrics  import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.77      0.80      0.90      0.79      0.85      0.72       962
          2       0.23      0.08      0.98      0.12      0.28      0.07       244
          3       0.27      0.10      0.97      0.15      0.31      0.09       347
          4       0.40      0.34      0.88      0.37      0.55      0.29       623
          5       0.64      0.88      0.74      0.74      0.81      0.66      1145

avg / total       0.56      0.62      0.85      0.58      0.68      0.50      3321



Pour confirmer notre observation, le score f1 et la moyenne géometrique des classes non extremes montrent bien que les données sont déséquilibrés.

Commençons par effectuer un sous échantillonnage: 

In [29]:
from imblearn.under_sampling import RandomUnderSampler

ru = RandomUnderSampler()
X_ru, y_ru = ru.fit_resample(X_train,y_train)

C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


puis appliquons notre model avec les parametres de notre premiere modélisation:

In [31]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_ru, y_ru)
y_pred = clf.predict(X_test)
display(pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"]))
print(clf.score(X_test,y_test))

C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,576,214,98,46,28
2,46,74,70,48,6
3,24,71,113,112,27
4,12,50,128,266,167
5,19,25,85,298,718


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.5260463715748268


Le sous echantillonnage à fortement diminué les performances de notre model. Essayons la démarche inverse avec le sur échantillonnage:

In [32]:
from imblearn.over_sampling import RandomOverSampler

ro = RandomUnderSampler()
X_ro, y_ro = ru.fit_resample(X_train,y_train)

C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [33]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_ro, y_ro)
y_pred = clf.predict(X_test)
display(pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"]))
print(clf.score(X_test,y_test))

C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,593,210,116,25,18
2,50,77,91,19,7
3,25,69,165,53,35
4,14,64,221,132,192
5,13,51,167,158,756


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.5188196326407708


Le sur echantillonnage a aussi diminué les performances de notre model. 

#### Observations: 
Le rééchantillonnage ne semble pas être la bonne méthode pour réduire les écarts dans les prédictions entres les valeurs extremes et les valeurs intermediaires

## Modelisation 2

Dans une seconde tentative de modélisation, nous allons inclure les méta données dans l'application de notre algorithme de gradient boosting.
#### La notions de date ne sera pas pris en compte pour des raisons de facilités.


In [12]:
df2 = df[["Commentaires_reduit_4mots","client","ville","reponse","source","company"]]
#la 1ere transformation ayant donnée des données de type booléén, nous changeons le type des deux colonnes suivantes pour
#ne conserver que des données numériques 
df2["source"] = df2["source"].astype("int64")
df2["company"] = df2["company"].astype("int64")
#suppression des valeurs nan de la colonne commentaires:
df2 = df2.dropna(axis = 0, how = "all", subset =["Commentaires_reduit_4mots"])
df2.head()

C:\Users\Hp\AppData\Local\Temp\ipykernel_17640\3100451435.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["source"] = df2["source"].astype("int64")
C:\Users\Hp\AppData\Local\Temp\ipykernel_17640\3100451435.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["company"] = df2["company"].astype("int64")


,Commentaires_reduit_4mots,client,ville,reponse,source,company
0,bonjour doit environ membre showrooprive jamai...,1,0,0,0,0
1,vente lacoste manquant photo prise articles mo...,1,0,0,0,0
2,vente lacoste honteuse erroné manquant livrais...,1,0,0,0,0
3,commandé mules marque moosefield produit étiqu...,1,0,1,0,0
4,téléphone etat livraison vieux téléphone pourr...,1,0,1,0,0


In [13]:
#vectorisation colonnes commentaires
vectorizer.fit_transform(freq_sup_100["index"])
tokenized = vectorizer.vocabulary_
Com_Matrix= vectorizer.transform(df2["Commentaires_reduit_4mots"]).todense()
Com_Matrix.shape

(16604, 540)

In [14]:
#transformation du dataframe des meta données en matrice
Meta_Matrix = df2.drop("Commentaires_reduit_4mots", axis =1).to_numpy()
Meta_Matrix.shape

(16604, 5)

In [15]:
#concatenation des matrices
X = np.concatenate([Com_Matrix,Meta_Matrix],axis=1)
X.shape

(16604, 545)

In [58]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

y_pred = clf.predict(X_test)
display(pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"]))
print(clf.score(X_test,y_test))

C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,843,23,15,50,31
2,73,26,35,84,26
3,28,25,41,185,68
4,24,5,39,305,250
5,38,3,6,128,970


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.657934357121349


L'inclusion des méta données semble améliorer la performance du model.

Combinons cette inclusion avec l'extension du nombre de mots vectorisés

In [16]:
#vectorisation des mots apparaissant plus de 50 fois dans les commentaires
freq = freqmots[freqmots["iteration"] >= 50].reset_index()
vectorizer.fit_transform(freq["index"])
tokenized = vectorizer.vocabulary_
Com_Matrix = vectorizer.transform(df1["Commentaires_reduit_4mots"]).todense()
Com_Matrix.shape

(16604, 959)

In [17]:
X = np.concatenate([Com_Matrix,Meta_Matrix],axis=1)
X.shape

(16604, 964)

In [65]:
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100)
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

y_pred = clf.predict(X_test)
display(pd.crosstab(y_test,y_pred, rownames =["valeurs réelle"] , colnames = ["valeurs prédites"]))
print(clf.score(X_test,y_test))

C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


valeurs prédites,1,2,3,4,5
valeurs réelle,,,,,
1,834,28,23,46,31
2,80,19,37,84,24
3,30,23,48,182,64
4,25,13,40,293,252
5,44,3,6,119,973


C:\Users\Michelle\anaconda3\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


0.652514302920807


## Comparaison du modèle du Gradient boosting avec le modèle SVM,Random Forest et régression logistique



### Modélisation Random Forest 



Nous allons appiquer l'algorithme de Random Forest en incluant les méta données et l'extension du nombre de mots vectorisés.

Appliquons la fonction GridSearchCV() du module model_selection pour obtenir les meilleurs paramètres du modèle Random Forest:

1ère application:

In [20]:
from sklearn import ensemble
from sklearn import model_selection
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100)

rf = ensemble.RandomForestClassifier()

grid_rf = model_selection.GridSearchCV(estimator=rf, param_grid={'max_features': ["sqrt", "log2", None ]})
grid_rf.fit(X_train, y_train)

print('Meilleurs paramètres :', grid_rf.best_params_)

y_pred = grid_rf.predict(X_test)
display(pd.crosstab(y_test, y_pred, rownames = ['Classe réelle'], colnames = ['Classe prédite par rf']))
print("Score :", grid_rf.score(X_test,y_test))




Meilleurs paramètres : {'max_features': 'log2'}


Classe prédite par rf,1,2,3,4,5
Classe réelle,,,,,
1,885,8,12,30,27
2,129,7,25,56,27
3,78,18,45,140,66
4,60,9,51,248,255
5,49,1,21,127,947


Score : 0.6419753086419753


2ème application:

In [21]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier(n_jobs=-1, random_state=300, max_features = 'log2')                                      

grid_rf = model_selection.GridSearchCV(estimator=rf, param_grid={'min_samples_split': range(2,40)})
grid_rf.fit(X_train, y_train)

print('Meilleurs paramètres :', grid_rf.best_params_)

y_pred = grid_rf.predict(X_test)
display(pd.crosstab(y_test, y_pred, rownames = ['Classe réelle'], colnames = ['Classe prédite par rf']))
print("Score :", grid_rf.score(X_test,y_test))

Meilleurs paramètres : {'min_samples_split': 12}


Classe prédite par rf,1,2,3,4,5
Classe réelle,,,,,
1,900,2,9,28,23
2,143,0,15,61,25
3,104,8,28,135,72
4,66,3,31,244,279
5,56,1,5,91,992


Score : 0.65161096055405


### Modelisation SVM



In [2]:
#from sklearn import svm
#from sklearn import model_selection
#X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100)
#svm_clf = svm.SVC(probability=True)
#parametres = {'C':[0.1,1,10],'kernel':['rbf', 'sigmoid', 'linear'], 'gamma':[0.01,0.1,0.5]}
#grid_svm = model_selection.GridSearchCV(estimator=svm_clf, param_grid=parametres)
#grid_svm.fit(X_train, y_train)
#print("Meilleurs paramètres :", grid_svm.best_params_)
#y_pred = grid_svm.predict(X_test)
#display(pd.crosstab(y_test, y_pred, rownames = ['Classe réelle'], colnames = ['Classe prédite par svm']))
#print(grid_svm.score(X_test, y_test))

Appliquons la fonction GridSearchCV() du module model_selection pour obtenir les meilleurs paramètres du modèle SVM:

1ère Application:

In [39]:
from sklearn import model_selection
from sklearn import ensemble
from sklearn import svm

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100)

svm_clf = svm.SVC()

parametres = {'C':[0.1,1,10,30]}
grid_svm = model_selection.GridSearchCV(estimator=svm_clf, param_grid=parametres)
grid_svm.fit(X_train, y_train)


print("Meilleurs paramètres :", grid_svm.best_params_)

print("Taux", grid_svm.score(X_test, y_test))

y_pred = grid_svm.predict(X_test)
display(pd.crosstab(y_test, y_pred, rownames = ['Classe réelle'], colnames = ['Classe prédite par svm']))


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\skl

Meilleurs paramètres : {'C': 1}


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Taux 0.6636555254441433


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classe prédite par svm,1,2,3,4,5
Classe réelle,,,,,
1,867,6,21,50,18
2,93,15,34,84,18
3,43,16,40,194,54
4,30,4,31,352,206
5,42,1,7,165,930


2ème application:

In [44]:
from sklearn import model_selection
from sklearn import ensemble
from sklearn import svm

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100)

svm_clf = svm.SVC(C=1)

parametres = {'kernel':['rbf', 'sigmoid', 'linear'], 'gamma':[0.01,0.1,0.5]}
grid_svm = model_selection.GridSearchCV(estimator=svm_clf, param_grid=parametres)
grid_svm.fit(X_train, y_train)


print("Meilleurs paramètres :", grid_svm.best_params_)

print("Taux", grid_svm.score(X_test, y_test))

y_pred = grid_svm.predict(X_test)
display(pd.crosstab(y_test, y_pred, rownames = ['Classe réelle'], colnames = ['Classe prédite par svm']))


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\Hp\anaconda3\lib\site-packages\skl

Meilleurs paramètres : {'gamma': 0.1, 'kernel': 'rbf'}


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Taux 0.6636555254441433


C:\Users\Hp\anaconda3\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classe prédite par svm,1,2,3,4,5
Classe réelle,,,,,
1,883,5,13,41,20
2,112,7,34,74,17
3,66,14,42,171,54
4,51,3,28,328,213
5,53,0,5,143,944


### Modelisation Regression Logistique

In [22]:
from sklearn import linear_model 
from sklearn import model_selection
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

X_train,X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 100)

rl = linear_model.LogisticRegression()

grid_rl = model_selection.GridSearchCV(estimator=rl, param_grid={'C': [0.001, 0.01, 0.1, 0.3, 1, 3, 10]})
grid_rl.fit(X_train, y_train)

print('Meilleurs paramètres :', grid_rl.best_params_)

y_pred = grid_rl.predict(X_test)
display(pd.crosstab(y_test, y_pred, rownames = ['Classe réelle'], colnames = ['Classe prédite par rl']))
print("Score :", grid_rl.score(X_test,y_test))


Meilleurs paramètres : {'C': 0.1}


Classe prédite par rl,1,2,3,4,5
Classe réelle,,,,,
1,848,16,26,43,29
2,79,29,37,72,27
3,28,29,54,172,64
4,21,7,45,311,239
5,25,0,6,133,981


Score : 0.6693766937669376


Conclusion : On obtient un score meilleur avec le modèle de régression logistique supérieur au score du modèle de gradient boosting

# Vectorization avec TfIdf

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import warnings
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [36]:
df = pd.read_csv('C:/Users/thier/Downloads/Data Satisfaction - Dan processed.csv')
df.head()

In [27]:
df = df.drop(["Unnamed: 0","Language"],axis =1)

df["client"] = df["client"].isna().replace(to_replace =[False,True], value=[1,0])
df["ville"] = df["ville"].isna().replace(to_replace =[False,True], value=[1,0])
df["reponse"] = df["reponse"].isna().replace(to_replace =[False,True], value=[1,0])

df["source"] = df["source"].isna().replace(to_replace =["TrustPilot","TrustedShop"], value=[1,0])
df["company"] = df["company"].isna().replace(to_replace =["ShowRoom","VeePee"], value=[1,0])

df1 = df[["star", "Commentaires_reduit_4mots"]]
df1.isna().sum()
df1 = df1.dropna(axis = 0, how = "all", subset =["Commentaires_reduit_4mots"])

df2 = df[["star", "Commentaires_reduit"]]
df2.isna().sum()
df2 = df2.dropna(axis = 0, how = "all", subset =["Commentaires_reduit"])

L'algorithme suivant utilise un filtre min_df = 5, qui reduit retire les mots a faible occurence.

##### Pour rappel min_df (par defaut 1, on ignore aucun terme)
Permet de ne pas considerer les termes (mots) apparaissant trop rarement.
Nous avons les 2 possibilites suivantes.
######     min_df = nombre entier
Ignore le termes apparaissant moins de  nombre entier de fois des documents.
######     min_df = nombre flotant (pourcentage)
Ignore le terme apparaissant moins de nombre flotant  dans tous les documents.
Exprime en pourcentage: ex 0.01 on ignore les termes qui apparaissent dans moins 1% des documents

##  Avec "Commentaires_reduit_4mots"

#### Initial 2 'notes' (df['Sentiment'] = (df1['star'] > 2).astype(int))

In [8]:
X = df1.Commentaires_reduit_4mots
df1['Sentiment'] = (df1['star'] > 2).astype(int)
y = df1.Sentiment

text_train, text_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tic= time.perf_counter()

vect = CountVectorizer(min_df=5)
X_train = vect.fit_transform(text_train)
X_test = vect.transform(text_test)

pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 0.3, 1, 3, 10]}

grid = GridSearchCV(pipe, param_grid, cv=5, refit=True, n_jobs=-1)
grid.fit(text_train, y_train)

toc = time.perf_counter()
print(f"Training around {round((toc - tic)//60)} min")

print("Best cross_validation score: {:.2f}".format(grid.best_score_))
print(" best estimator : ", grid.best_estimator_)
print(" best params : ", grid.best_params_)



Training around 0 min
Best cross_validation score: 0.89
 best estimator :  Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(min_df=5)),
                ('logisticregression', LogisticRegression(C=3))])
 best params :  {'logisticregression__C': 3}


#### 5 'notes' (df1['Sentiment'] = (df1['star']).astype(int))

In [30]:
X = df1.Commentaires_reduit_4mots
df1['Sentiment'] = (df1['star']).astype(int)
y = df1.Sentiment

text_train, text_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tic= time.perf_counter()

vect = CountVectorizer(min_df=5)
X_train = vect.fit_transform(text_train)
X_test = vect.transform(text_test)

pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 0.3, 1, 3, 10]}

grid = GridSearchCV(pipe, param_grid, cv=5, refit=True, n_jobs=-1)
grid.fit(text_train, y_train)

toc = time.perf_counter()
print(f"Training around {round((toc - tic)//60)} min")

print("Best cross_validation score: {:.2f}".format(grid.best_score_))
print(" best estimator : ", grid.best_estimator_)
print(" best params : ", grid.best_params_)

Training around 0 min
Best cross_validation score: 0.66
 best estimator :  Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(min_df=5)),
                ('logisticregression', LogisticRegression(C=1))])
 best params :  {'logisticregression__C': 1}


Le passage de 2 a 5 valeurs de sentiment reduit le Best cross_validation score, de 0.89 a 0.66.
Les meilleurs parametres pour la logistiqye regression evoluent (C de 3 a 1). 

##  Avec "Commentaires_reduit"

In [11]:
df2 = df[["star", "Commentaires_reduit"]]
df2.isna().sum()
df2 = df2.dropna(axis = 0, how = "all", subset =["Commentaires_reduit"])

#### Initial 2 'notes' (df2['Sentiment'] = (df2['star'] > 2).astype(int))

In [14]:
X = df2.Commentaires_reduit
df2['Sentiment'] = (df2['star'] > 2).astype(int)
y = df2.Sentiment

text_train, text_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tic= time.perf_counter()

vect = CountVectorizer(min_df=5)
X_train = vect.fit_transform(text_train)
X_test = vect.transform(text_test)

pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 0.3, 1, 3, 10]}

grid = GridSearchCV(pipe, param_grid, cv=5, refit=True, n_jobs=-1)
grid.fit(text_train, y_train)

toc = time.perf_counter()
print(f"Training around {round((toc - tic)//60)} min")

print("Best cross_validation score: {:.2f}".format(grid.best_score_))
print(" best estimator : ", grid.best_estimator_)
print(" best params : ", grid.best_params_)

Training around 0 min
Best cross_validation score: 0.90
 best estimator :  Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(min_df=5)),
                ('logisticregression', LogisticRegression(C=3))])
 best params :  {'logisticregression__C': 3}


Le passage de "Commentaires_reduit_4mots" a  "Commentaires_reduit" comme jeu de donnees initiales, augmente le resultat Best cross_validation score de 0.89 a 0.90 (significatif ?).
Les meilleurs parametres pour la logistiqye regression restent inchanges. 

#### Initial 5 'notes' (df2['Sentiment'] = (df2['star'] > 2).astype(int))

In [15]:
X = df2.Commentaires_reduit
df2['Sentiment'] = (df2['star']).astype(int)
y = df2.Sentiment

text_train, text_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tic= time.perf_counter()

vect = CountVectorizer(min_df=5)
X_train = vect.fit_transform(text_train)
X_test = vect.transform(text_test)

pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 0.3, 1, 3, 10]}

grid = GridSearchCV(pipe, param_grid, cv=5, refit=True, n_jobs=-1)
grid.fit(text_train, y_train)

toc = time.perf_counter()
print(f"Training around {round((toc - tic)//60)} min")

print("Best cross_validation score: {:.2f}".format(grid.best_score_))
print(" best estimator : ", grid.best_estimator_)
print(" best params : ", grid.best_params_)

Training around 0 min
Best cross_validation score: 0.67
 best estimator :  Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer(min_df=5)),
                ('logisticregression', LogisticRegression(C=1))])
 best params :  {'logisticregression__C': 1}


Le passage de 2 a 5 valeurs de sentiment reduit le Best cross_validation score, de 0.90 a 0.67.
Les meilleurs parametres pour la logistiqye regression evoluent (C de 3 a 1), comme precedemment. 

### Synthese

Le passage de 2 niveau de sentiment (star) a 5 reduite fortement les performances.
L'utilisation de commentaire reduit au lieu de commentaire reduit a au moins 4 mots, ameliore faiblement le score (0.67 / 0.66).